**Instituto Federal de Minas Gerais - Campus Ouro Preto**

**Disciplina:** Recuperação da Informação

**Professor:** Moisés

**Aluno:** Marco Antonio do Nascimento

**Orientações da atividade:** Seguindo a abordagem do código feito durante a aula expositiva, ficando como sugestão apenas, você deve construir seu próprio coletor. Você deve definir um contexto que você deseja atuar para tratamento e busca de informações em seu trabalho. A partir de uma lista de urls para esse contexto, você vai implementar o seu coletor de forma sequencial para a lista fornecida, bem como propor algum tipo de aprofundamento recursivo dos links de cada página coletada (profundidade ou em largura até um nível k, à sua escolha).
Você deve também implementar quais tags de texto você vai buscar para realizar depois a indexação no próximo trabalho. Em nossa aula, usamos apenas a tag "h2" mas não é só ela que possui os textos das páginas.
Apenas como complemento, você deve estudar (ou até implementar) alguma forma para tratar o redirecionamento de páginas.
O seu Sistema de RI aqui pode ter apenas as classes envolvidas para a coleta (em nossa aula, seriam Url e Coletor) e um arquivo principal para acionar os respectivos objetos. Não se esqueça das buscas em profundidade ou em largura, que não foram tratadas (ainda) nas aulas.
Com tudo pronto, apenas submeta seu arquivo em pasta compactada para esta tarefa.

A ideia deste trabalho é extrair dados que estão sendo visualizados do site Fundamentus. Tal site mostra os principais indicadores das ações da B3 de forma automatizada. Para isso, será usado a técnica de WebScraping.
O primeiro passo para isso é instalar a biblioteca "bs4" (beautiful soap).  

In [10]:
!pip install bs4


Isso feito, faremos as importações abaixo necessárias, inserimos o site do fundamentus (url) e também o comando headers, para conseguir obter o HTML utilizando o python. Então esse código realiza as operações básicas de Web Scraping para obter o HTML de uma página da web, extrair informações relevantes e analisá-las com BeautifulSoup para posterior processamento.


In [11]:
#Importando bibliotecas
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
 
 
#Obtendo o HTML
url = "https://www.fundamentus.com.br/resultado.php"
headers ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}
 
req = Request(url,headers=headers)
response = urlopen(req)
html = response.read()
soup = BeautifulSoup(html,'html.parser')
 
#Obtendo as TAGs de interesse ( topo das informações)
lista =soup.find('table')

O próximo passo é extrair a quantidade de ocorrências de determinadas tags HTML e definir uma lista vazia para armazenar informações resumidas ou agregadas. Essas etapas são preparatórias para a extração de informações específicas que serão realizadas posteriormente no código.






In [12]:
#quantidade ações
qtd = soup.findAll('span',class_='tips') 
qtd = range(int(len(qtd)-1)) #retira o ultimo registro para não ter erro
 
#Declarando variáveis cards
resumo = []


Agora precisamos percorrer as linhas da tabela HTML e extrair as informações específicas de cada linha, armazenando-as em um dicionário e, em seguida, agregando esses dicionários em uma lista. Nesse momento, conseguimos localizar as informações das ações que estão sendo observadas, linha a linha. Neste sentido, precisei inserir manualmente o nome das colunas, como pode ser visto no comando abaixo:


In [13]:
#pega as primeiras informações que não entram no for
papel =  lista.find('td').find('span',class_='tips').getText()
cotacao = lista.find('td').findNext('td').contents[0]
 
for i in qtd:
 
  acoes ={}
 
  PL = cotacao.findNext('td').contents[0]
  PVP = PL.findNext('td').contents[0]
  PSR = PVP.findNext('td').contents[0]
  DividendYied = PSR.findNext('td').contents[0]
  PAtivo = DividendYied.findNext('td').contents[0]
  PCapGiro = PAtivo.findNext('td').contents[0]
  PEbit= PCapGiro.findNext('td').contents[0]
  PAtivoCirc= PEbit.findNext('td').contents[0]
  EVEbit= PAtivoCirc.findNext('td').contents[0]
  EVEbita= EVEbit.findNext('td').contents[0]
  MrgEbit= EVEbita.findNext('td').contents[0]
  MrgLiq= MrgEbit.findNext('td').contents[0]
  LiqCorrente= MrgLiq.findNext('td').contents[0]
  ROIC= LiqCorrente.findNext('td').contents[0]
  ROE= ROIC.findNext('td').contents[0]
  Liq2Meses= ROE.findNext('td').contents[0]
  PatriLiquido= Liq2Meses.findNext('td').contents[0]
  DivBruta_por_Patri= PatriLiquido.findNext('td').contents[0]
  Cresc_5a= DivBruta_por_Patri.findNext('td').contents[0]
 
  acoes['id']= i
  acoes['Papel'] = papel
  acoes['Cotacao'] = cotacao
  acoes['PL'] = PL
  acoes['PVP']=PVP
  acoes['DividendYied']=DividendYied
  acoes['PAtivo']=PAtivo
  acoes['PCapGiro']=PCapGiro
  acoes['PEbit']=PEbit
  acoes['PAtivoCirc']=PAtivoCirc
  acoes['EVEbit']=EVEbit
  acoes['EVEbita']=EVEbita
  acoes['MrgEbit']=MrgEbit
  acoes['MrgLiq']=MrgLiq
  acoes['LiqCorrente']=LiqCorrente
  acoes['ROIC']=ROIC
  acoes['ROE']=ROE
  acoes['Liq2Meses']=Liq2Meses
  acoes['PatriLiquido']=PatriLiquido
  acoes['DivBruta_por_Patri']=DivBruta_por_Patri
  acoes['Cresc_5a']=Cresc_5a
 
  #Adiciona o dicionário de ações em uma lista
  resumo.append(acoes)
 
  #try retorna erro por a ultima linha não encontra o spam
  try:
    papel = Cresc_5a.findNext('td').span.a.contents[0]
    cotacao = papel.findPrevious('td').findNext('td').contents[0]
 
  except HTTPError as e:
    print(e.status, e.reason)
 

No trecho abaixo do código, um DataFrame do Pandas está sendo criado usando a lista resumo, que contém os dicionários de ações que foram extraídos da tabela HTML. Cada elemento da lista resumo é tratado como uma linha no DataFrame.
Isso significa que o código está criando um DataFrame do Pandas contendo os dados das ações extraídas da tabela HTML e exibindo as primeiras 10000 linhas desse DataFrame.

In [14]:
#cria data frame
dataset = pd.DataFrame(resumo)
 
dataset.head(10000)

,id,Papel,Cotacao,PL,PVP,DividendYied,PAtivo,PCapGiro,PEbit,PAtivoCirc,...,EVEbita,MrgEbit,MrgLiq,LiqCorrente,ROIC,ROE,Liq2Meses,PatriLiquido,DivBruta_por_Patri,Cresc_5a
0,0,POPR4,"10,17","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","8,66%","5,65%","1,08","15,25%","19,93%","0,00","545.803.000,00","0,82","30,93%"
1,1,CSTB3,"150,00","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","40,85%","28,98%","2,60","22,40%","20,11%","0,00","8.420.670.000,00","0,14","31,91%"
2,2,CSTB4,"147,69","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","40,85%","28,98%","2,60","22,40%","20,11%","0,00","8.420.670.000,00","0,14","31,91%"
3,3,PMET3,"0,00","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","4,10%","0,00","-290.863.000,00","0,00","37,74%"
4,4,PORP4,"2,40","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","-2,08%","0,00","22.399.000,00","0,00","13,66%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,980,UBBR4,"7,49","610,27","1,99","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","0,33%","0,00","10.317.200.000,00","0,00","10,58%"
981,981,UBBR11,"14,75","1.201,81","3,91","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","0,33%","0,00","10.317.200.000,00","0,00","10,58%"
982,982,CEAB3,"9,70","1.280,50","0,99","0,00%","0,317","2,41","5,41","-1,52",...,"2,58","8,22%","0,03%","1,39","8,47%","0,08%","34.122.200,00","3.012.720.000,00","0,56","9,36%"
983,983,UBBR3,"18,00","1.466,61","4,77","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","0,33%","0,00","10.317.200.000,00","0,00","10,58%"


Cabe agora fazer o tratamento dos dados com o objetivo de garantir que os valores nos campos sejam do tipo correto e estejam prontos para análise. Isso inclui a remoção de caracteres não numéricos, como pontos e vírgulas, e a conversão dos valores para o tipo numérico apropriado.

In [15]:
#TRATAMENTO DE DADOS CAMPO PL  
dataset['PL'] = dataset['PL'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'PL': float}
dataset['PL']  = dataset['PL'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO ROE
dataset['ROE'] = dataset['ROE'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'ROE': float}
dataset['ROE']  = dataset['ROE'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO MrgLiq
dataset['MrgLiq'] = dataset['MrgLiq'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'MrgLiq': float}
dataset['MrgLiq']  = dataset['MrgLiq'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO DIVIDA BRUTA/PATRIMONIO  
dataset['DivBruta_por_Patri'] = dataset['DivBruta_por_Patri'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'DivBruta_por_Patri': float}
dataset['DivBruta_por_Patri']  = dataset['DivBruta_por_Patri'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO CAGER
dataset['Cresc_5a'] = dataset['Cresc_5a'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'Cresc_5a': float}
dataset['Cresc_5a']  = dataset['Cresc_5a'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS DIVIDEND YIED
dataset['DividendYied'] = dataset['DividendYied'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'DividendYied': float}
dataset['DividendYied']  = dataset['DividendYied'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO ROIC
dataset['ROIC'] = dataset['ROIC'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'ROIC': float}
dataset['ROIC']  = dataset['ROIC'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO ROIC
dataset['MrgEbit'] = dataset['MrgEbit'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'MrgEbit': float}
dataset['MrgEbit']  = dataset['MrgEbit'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO PVP  
dataset['PVP'] = dataset['PVP'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'PVP': float}
dataset['PVP']  = dataset['PVP'].astype(convert_dict)
 
 
#TRATAMENTO DE DADOS CAMPO EVEbit
dataset['EVEbit'] = dataset['EVEbit'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'EVEbit': float}
dataset['EVEbit']  = dataset['EVEbit'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO EVEbita
dataset['EVEbita'] = dataset['EVEbita'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'EVEbita': float}
dataset['EVEbita']  = dataset['EVEbita'].astype(convert_dict)

Por fim, basta agora classificar os dados de acordo com os critérios desejados. No meu caso, optei por classificar em ordem descresente as empresas que possuem o melhor preço/lucro, sendo que o ROE precisa estar entre 0 e 90; margem liquida >0, dívida bruta/patrimonio e crescimento da receita em 5 anos maior que 0,1.  
Assim, conseguimos filtrar as ações de uma forma que o próprio site não consegue nos fornecer, dado que o site apenas consegue ordenar de forma decrescente uma coluna por vez. 

In [16]:
#Blacklist de ações descontinuadas
blacklist ={
    'PTPA3'
}
 
#Filtragem de dados
selecao = (dataset['PL'] >= 1) & (dataset['ROE'] > 0) & (dataset['ROE'] < 90) &  (dataset['MrgLiq'] > 0) & (dataset['DivBruta_por_Patri'] > 1.3) &  (dataset['Cresc_5a'] > 0.1)
 
melhores_acoes = dataset[selecao].sort_values('PL', ascending=False)
melhores_acoes

,id,Papel,Cotacao,PL,PVP,DividendYied,PAtivo,PCapGiro,PEbit,PAtivoCirc,...,EVEbita,MrgEbit,MrgLiq,LiqCorrente,ROIC,ROE,Liq2Meses,PatriLiquido,DivBruta_por_Patri,Cresc_5a
984,984,CEPE3,"128,00",1591.72,6.04,0.0,"0,695","29,30","7,83","-1,13",...,10.37,0.1440,0.0007,"1,10",0.1001,0.0038,"0,00","1.580.000.000,00",5.15,3.3155
979,979,CEPE6,"43,00",534.72,2.03,0.0,"0,233","9,84","2,63","-0,38",...,6.48,0.1440,0.0007,"1,10",0.1001,0.0038,"0,00","1.580.000.000,00",5.15,3.3155
978,978,CEPE5,"40,12",498.91,1.89,0.0,"0,218","9,18","2,45","-0,35",...,6.35,0.1440,0.0007,"1,10",0.1001,0.0038,"0,00","1.580.000.000,00",5.15,3.3155
969,969,MYPK4,"31,79",159.01,1.32,0.0,"0,336","1,69","6,51","-1,67",...,7.12,0.0502,0.0071,"1,63",0.0777,0.0083,"0,00","3.693.630.000,00",1.83,0.1574
968,968,HBSA3,"3,63",156.84,2.00,0.0,"0,442","4,78","7,29","-0,78",...,8.61,0.1967,0.0091,"1,78",0.0719,0.0127,"14.643.000,00","1.383.520.000,00",3.11,0.1767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,464,PTPA4,"80,00",3.86,1.19,0.0,"0,392","2,38","3,79","-1,33",...,5.45,0.1393,0.1369,"1,79",0.1327,0.3085,"0,00","1.201.690.000,00",1.59,0.1808
455,455,PTPA3,"76,50",3.69,1.14,0.0,"0,375","2,28","3,63","-1,27",...,5.33,0.1393,0.1369,"1,79",0.1327,0.3085,"0,00","1.201.690.000,00",1.59,0.1808
456,456,REDE4,"2,90",3.69,1.52,0.0,"0,221","4,40","1,46","-0,45",...,3.19,0.2605,0.1424,"1,21",0.1775,0.4122,"0,00","4.022.100.000,00",3.09,0.1075
432,432,SUZB5,"21,13",1.96,0.62,0.0,"0,192","1,16","2,71","-0,46",...,4.78,0.2555,0.3548,"2,61",0.0869,0.3151,"0,00","44.692.800.000,00",1.73,0.1435


In [21]:
# Para ordenar as ações pelo ROE, por exemplo, basta usar o comando:
melhor_ROE = dataset[selecao].sort_values('ROE', ascending=False)
melhor_ROE

,id,Papel,Cotacao,PL,PVP,DividendYied,PAtivo,PCapGiro,PEbit,PAtivoCirc,...,EVEbita,MrgEbit,MrgLiq,LiqCorrente,ROIC,ROE,Liq2Meses,PatriLiquido,DivBruta_por_Patri,Cresc_5a
475,475,CPFP4,"44,00",4.30,3.95,0.0000,"0,300","-15,31","2,06","-0,48",...,4.19,0.1970,0.0941,"0,94",0.1763,0.9169,"0,00","592.154.000,00",6.38,0.1030
901,901,CGAS3,"106,00",39.93,31.62,0.0298,"1,003","56,99","4,88","-1,71",...,5.65,0.1808,0.0221,"1,05",0.2771,0.7918,"11.012,20","444.316.000,00",17.38,0.2006
910,910,CGAS5,"114,39",43.09,34.12,0.0303,"1,082","61,50","5,27","-1,85",...,5.97,0.1808,0.0221,"1,05",0.2771,0.7918,"161.214,00","444.316.000,00",17.38,0.2006
712,712,BEEF3,"7,16",11.91,6.92,0.0769,"0,168","0,62","2,17","-0,45",...,5.28,0.0727,0.0127,"1,87",0.1588,0.5814,"61.175.800,00","627.948.000,00",30.93,0.1636
533,533,EEEL4,"410,00",6.49,2.94,0.0000,"0,679","5,21","3,84","-1,27",...,5.41,0.5593,0.3322,"2,28",0.1978,0.4530,"0,00","1.336.810.000,00",1.59,0.1058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,942,MYPK3,"14,23",71.18,0.59,0.0062,"0,151","0,76","2,91","-0,75",...,4.91,0.0502,0.0071,"1,63",0.0777,0.0083,"11.591.100,00","3.693.630.000,00",1.83,0.1574
969,969,MYPK4,"31,79",159.01,1.32,0.0000,"0,336","1,69","6,51","-1,67",...,7.12,0.0502,0.0071,"1,63",0.0777,0.0083,"0,00","3.693.630.000,00",1.83,0.1574
978,978,CEPE5,"40,12",498.91,1.89,0.0000,"0,218","9,18","2,45","-0,35",...,6.35,0.1440,0.0007,"1,10",0.1001,0.0038,"0,00","1.580.000.000,00",5.15,3.3155
979,979,CEPE6,"43,00",534.72,2.03,0.0000,"0,233","9,84","2,63","-0,38",...,6.48,0.1440,0.0007,"1,10",0.1001,0.0038,"0,00","1.580.000.000,00",5.15,3.3155
